In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import mediapipe as mp
from pathlib import Path  # for safer path joins if needed
import pandas as pd

# access utils if using shared code
import sys
sys.path.append("..")

In [2]:
# helper function
def extract_eye_roi(landmarks, indices, w, h):
    points = []
    for idx in indices:
        try:
            # scale normalized landmark coordinates to pixel dimensions
            x = int(landmarks.landmark[idx].x * w)
            y = int(landmarks.landmark[idx].y * h)
            points.append((x, y))
        except IndexError:
            # skip if landmark index is missing
            continue
    # return points only if enough are found to form a reliable bounding box
    return points if len(points) >= 6 else None

In [3]:
# helper function to crop, resize, and save an eye ROI
def crop_and_save(img, points, filename, output_path, side):
    # unpack x and y coordinates
    x_coords, y_coords = zip(*points)
    # get bounding box coordinates with safety clamps
    x_min, x_max = max(min(x_coords), 0), min(max(x_coords), img.shape[1])
    y_min, y_max = max(min(y_coords), 0), min(max(y_coords), img.shape[0])

    # skip if bounding box is too small to be meaningful
    if (x_max - x_min < 10) or (y_max - y_min < 10):
        return False

    # crop the region of interest
    roi = img[y_min:y_max, x_min:x_max]
    # Resize to 224x224 to match CNN input
    roi_resized = cv2.resize(roi, (224, 224))
    # save with prefix indicating which eye
    save_path = os.path.join(output_path, f"{side}_{filename}")
    cv2.imwrite(save_path, roi_resized)
    return True  # return success flag

In [4]:
# Main function to extract eye ROIs from images in a folder
def roi_crop(input_path, output_path, label=None):
    # Initialize MediaPipe FaceMesh
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=True,       # Process each image independently
        refine_landmarks=True,        # Gives iris and more accurate eye contours
        max_num_faces=1,              # Only detect one face per image
        min_detection_confidence=0.5  # Increase if you're getting false detections
    )

    # Define landmark indices for left and right eyes
    LEFT_EYE_LANDMARKS = [463, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382, 362]
    RIGHT_EYE_LANDMARKS = [33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7]

    # For logging skipped files and why
    skipped_files = []

    # Loop through all files in the input directory
    for file in os.listdir(input_path):
        img_path = os.path.join(input_path, file)

        try:
            # Load the image using OpenCV
            img = cv2.imread(img_path)

            # Skip if image is unreadable or not 3-channel
            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                raise Exception("Invalid image format or unreadable image")

            # Convert BGR to RGB for MediaPipe
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(img_rgb)

            # If no face is detected, log and skip
            if not results.multi_face_landmarks:
                skipped_files.append((file, "No face detected"))
                continue

            # Get image dimensions
            h, w, _ = img.shape
            landmarks = results.multi_face_landmarks[0]

            # Extract landmark points for each eye
            left_eye_points = extract_eye_roi(landmarks, LEFT_EYE_LANDMARKS, w, h)
            right_eye_points = extract_eye_roi(landmarks, RIGHT_EYE_LANDMARKS, w, h)

            # Track if either eye crop was successful
            success = False

            # Crop left eye if possible
            if left_eye_points:
                success |= crop_and_save(img, left_eye_points, file, output_path, "left")

            # Crop right eye if possible
            if right_eye_points:
                success |= crop_and_save(img, right_eye_points, file, output_path, "right")

            # If neither eye was valid, log the failure
            if not success:
                skipped_files.append((file, "No valid eye ROI found"))

        except Exception as e:
            # Catch and log any unexpected errors
            skipped_files.append((file, f"Processing error: {e}"))

    # Return the list of all files that were skipped with reasons
    return skipped_files

In [5]:
processed_healthy_path = '../data/processed/healthy_eye'
processed_infected_path = '../data/processed/infected_eye'

roi_healthy_path = '../data/roi/healthy_eye'
roi_infected_path = '../data/roi/infected_eye'

# Run the cropper
skipped_healthy_roi = roi_crop(processed_healthy_path, roi_healthy_path, "Healthy")
skipped_infected_roi = roi_crop(processed_infected_path, roi_infected_path, "Infected")

# Combine skipped logs
skipped_roi = skipped_healthy_roi + skipped_infected_roi

I0000 00:00:1750632998.449037 23620984 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1750632998.457418 23640978 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750632998.462962 23640978 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750632998.473054 23640978 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
I0000 00:00:1750632998.988977 23620984 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Max
W0000 00:00:1750632998.989951 23641034 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature infe

In [6]:
import pandas as pd

if skipped_roi:
    df_skipped = pd.DataFrame(skipped_roi, columns=["filename", "reason"])
    df_skipped.to_csv("../data/log/skipped_roi_images.csv", index=False)
    print(f"Logged {len(skipped_roi)} skipped files to '../data/log/skipped_roi_images.csv'")
else:
    print("All ROI crops successful.")

Logged 291 skipped files to '../data/log/skipped_roi_images.csv'
